# Setting Up

## Installation

In [ ]:
%%capture
!pip install git+https://github.com/munir2200963/TTS.git

!sudo apt-get install -y espeak libsndfile1

  Cloning https://github.com/munir2200963/TTS.git to /tmp/pip-req-build-kwi870zs
  Running command git clone --filter=blob:none --quiet https://github.com/munir2200963/TTS.git /tmp/pip-req-build-kwi870zs
  Resolved https://github.com/munir2200963/TTS.git to commit 80e5c85f8e37ea96529400dcbdb5be2192862ff6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 37.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 67.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
%%capture
! wget https://coqui.gateway.scarf.sh/v0.7.0_models/tts_models--en--blizzard2013--capacitron-t2-c50.zip

! unzip '*.zip'

--2025-03-04 02:26:53--  https://coqui.gateway.scarf.sh/v0.7.0_models/tts_models--en--blizzard2013--capacitron-t2-c50.zip
Resolving coqui.gateway.scarf.sh (coqui.gateway.scarf.sh)... 54.244.195.224, 34.213.189.139, 52.33.86.107
Connecting to coqui.gateway.scarf.sh (coqui.gateway.scarf.sh)|54.244.195.224|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/coqui-ai/TTS/releases/download/v0.7.0_models/tts_models--en--blizzard2013--capacitron-t2-c50.zip [following]
--2025-03-04 02:26:53--  https://github.com/coqui-ai/TTS/releases/download/v0.7.0_models/tts_models--en--blizzard2013--capacitron-t2-c50.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/265612440/a6d29072-7586-40e6-97d4-6638dc6e9ca3?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Cred

## Imports and Initialisation of Functions

In [ ]:
import os
import sys
import io
import uuid
import contextlib
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm

import TTS
from TTS.utils.synthesizer import Synthesizer
from TTS.tts.utils.synthesis import synthesis
from TTS.config import load_config
from TTS.tts.models import setup_model as setup_tts_model

In [ ]:
# config.json and model_file.pth
MODEL_PATH_C50 = "tts_models--en--blizzard2013--capacitron-t2-c50/"

In [ ]:
def count_lines_in_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return sum(1 for _ in file)
    except FileNotFoundError:
        return "File not found"
    except Exception as e:
        return f"Error: {e}"

def print_summary(audio_folder, text_file, category):
    audio_count = len(os.listdir(audio_folder)) if os.path.exists(audio_folder) else "Folder not found"
    text_lines = count_lines_in_file(text_file)
    print(f"{category:<30} | Audio Files: {audio_count:<6} | Text Lines: {text_lines}")

In [ ]:
%%capture
def load_tts_model(tts_checkpoint: str, tts_config_path: str, use_cuda: bool = True):
    # Load the TTS configuration from the JSON file.
    tts_config = load_config(tts_config_path)

    # Initialize the model using the configuration.
    tts_model = setup_tts_model(config=tts_config)

    # Load the model weights from the checkpoint.
    tts_model.load_checkpoint(tts_config, tts_checkpoint, eval=True)

    # Move the model to GPU if requested.
    if use_cuda and torch.cuda.is_available():
        tts_model.cuda()

    return tts_model, tts_config

In [ ]:
def getProsodyEmbedding(audio_path: str, text: str) -> np.ndarray:
    """
    Extract the prosody embedding from a trained Capacitron-based TTS model,
    given a reference audio path and the corresponding reference text.

    Args:
        audio_path (str): Path to the reference audio (used as style input).
        text (str): The transcript of the reference audio.

    Returns:
        np.ndarray: A 1D NumPy array containing the prosody embedding.
    """
    # Adjust these paths for your environment
    model_dir = MODEL_PATH_C50
    tts_checkpoint = model_dir + "model_file.pth"
    tts_config_path = model_dir + "config.json"
    use_cuda = False

    # Load the TTS model
    model, config = load_tts_model(
        tts_checkpoint=tts_checkpoint,
        tts_config_path=tts_config_path,
        use_cuda=use_cuda
    )

    # Synthesize voice using the reference information (style_wav & style_text)
    outputs = synthesis(
        model=model,
        text="hello",
        CONFIG=config,
        use_cuda=use_cuda,
        speaker_id=None,
        style_wav=audio_path,
        style_text=text,
        use_griffin_lim=None,
        d_vector=None,
        language_id=None,
    )

    # Retrieve the Capacitron VAE outputs
    capacitron_vae_outputs = outputs["outputs"]["capacitron_vae_outputs"]

    # 1) Posterior distribution
    posterior_dist = capacitron_vae_outputs[0]  # MultivariateNormal with shape [1, 128]

    # Get the embedding (mean) from the posterior
    posterior_mean = posterior_dist.loc  # shape: [1, 128]

    # Convert to NumPy array
    posterior_mean_numpy = posterior_mean.squeeze(0).detach().cpu().numpy()

    return posterior_mean_numpy


## Extract Prosody Embeddings

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define the dataset folders
folders = {
    "Dev Enroll": "/content/drive/MyDrive/INF2008_Machine_Learning/B3/data/libri_dev_enrolls_B3",
    "Dev Male Trial": "/content/drive/MyDrive/INF2008_Machine_Learning/B3/data/libri_dev_trials_m_B3",
    "Dev Female Trial": "/content/drive/MyDrive/INF2008_Machine_Learning/B3/data/libri_dev_trials_f_B3",
    "Test Enroll": "/content/drive/MyDrive/INF2008_Machine_Learning/B3/data/libri_test_enrolls_B3",
    "Test Male Trial": "/content/drive/MyDrive/INF2008_Machine_Learning/B3/data/libri_test_trials_m_B3",
    "Test Female Trial": "/content/drive/MyDrive/INF2008_Machine_Learning/B3/data/libri_test_trials_f_B3",
}

# Map each category to its designated Tacotron embedding folder
tacotron_folders = {
    "Dev Enroll": "/content/drive/MyDrive/INF2008_Machine_Learning/B3/embeddings/Tacotron/Tacotron_dev_enroll_embeddings",
    "Dev Male Trial": "/content/drive/MyDrive/INF2008_Machine_Learning/B3/embeddings/Tacotron/Tacotron_dev_trial_m_embeddings",
    "Dev Female Trial": "/content/drive/MyDrive/INF2008_Machine_Learning/B3/embeddings/Tacotron/Tacotron_dev_trial_f_embeddings",
    "Test Enroll": "/content/drive/MyDrive/INF2008_Machine_Learning/B3/embeddings/Tacotron/Tacotron_test_enroll_embeddings",
    "Test Male Trial": "/content/drive/MyDrive/INF2008_Machine_Learning/B3/embeddings/Tacotron/Tacotron_test_trial_m_embeddings",
    "Test Female Trial": "/content/drive/MyDrive/INF2008_Machine_Learning/B3/embeddings/Tacotron/Tacotron_test_trial_f_embeddings",
}

# Ensure the Tacotron folders exist; if not, create them.
for tacotron_folder in tacotron_folders.values():
    os.makedirs(tacotron_folder, exist_ok=True)

print("\nSummary of Audio Files and Text Line Counts:\n")
print("-" * 80)
print(f"{'Category':<30} | {'Audio Files':<12} | {'Text Lines'}")
print("-" * 80)

for category, folder in folders.items():
    audio_folder = os.path.join(folder, "wav")
    text_file = os.path.join(folder, "text")
    print_summary(audio_folder, text_file, category)

print("-" * 80)


Summary of Audio Files and Text Line Counts:

--------------------------------------------------------------------------------
Category                       | Audio Files  | Text Lines
--------------------------------------------------------------------------------
Dev Enroll                     | Audio Files: 343    | Text Lines: 343
Dev Male Trial                 | Audio Files: 960    | Text Lines: 960
Dev Female Trial               | Audio Files: 1018   | Text Lines: 1018
Test Enroll                    | Audio Files: 438    | Text Lines: 438
Test Male Trial                | Audio Files: 762    | Text Lines: 762
Test Female Trial              | Audio Files: 734    | Text Lines: 734
--------------------------------------------------------------------------------


In [ ]:
# ------------------------------------------------------------------------------
# Process each dataset category to extract and save prosody embeddings.
#
# For each category (e.g., Dev Enroll, Dev Male Trial, etc.), the script:
# 1. Reads the corresponding text file which contains lines with an utterance ID
#    and its transcription text.
# 2. For each utterance, constructs the path to the corresponding WAV file (assumed
#    to be in a "wav" subfolder and named after the utterance ID, with a .wav extension).
# 3. Extracts the prosody embedding by calling the predefined getProsodyEmbedding function,
#    which takes the WAV file path and the utterance text as inputs and returns a NumPy array.
# 4. Saves the resulting embedding as a .npy file in the designated Tacotron output folder.
# 5. Uses TQDM to display a progress bar for processing each category.
# ------------------------------------------------------------------------------

for category, base_folder in folders.items():
    # Define the path to the text file and the corresponding Tacotron output folder
    text_file_path = os.path.join(base_folder, "text")
    tacotron_output_folder = tacotron_folders[category]

    # Open the text file and read all lines
    with open(text_file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    # Process each line with a TQDM progress bar
    for line in tqdm(lines, desc=f"Processing {category}", unit=" lines"):
        line = line.strip()
        if not line:
            continue
        try:
            # Split the line into utterance ID and utterance text.
            utterance_id, utterance_text = line.split(maxsplit=1)
        except ValueError:
            tqdm.write(f"Skipping line (could not split properly): {line}")
            continue

        # Construct the path to the WAV file (assumes file is named <utterance_id>.wav)
        utterance_wav_path = os.path.join(base_folder, "wav", utterance_id) + ".wav"

        # Use tqdm.write instead of print to avoid messing up the progress bar
        tqdm.write(f"Extracting prosody embedding for {utterance_id} in {category} from {utterance_wav_path}")

        # Capture any unwanted output/warnings from getProsodyEmbedding
        with io.StringIO() as buf, contextlib.redirect_stdout(buf), contextlib.redirect_stderr(buf):
            embedding = getProsodyEmbedding(utterance_wav_path, utterance_text)

        # Save the NumPy array as <utterance_id>.npy in the designated Tacotron folder
        output_file_path = os.path.join(tacotron_output_folder, utterance_id + ".npy")
        np.save(output_file_path, embedding)


Processing Dev Enroll:   0%|          | 0/343 [00:00<?, ? lines/s]

Extracting prosody embedding for 1272-128104-0000 in Dev Enroll from /content/drive/MyDrive/INF2008_Machine_Learning/B3/data/libri_dev_enrolls_B3/wav/1272-128104-0000.wav


Processing Dev Enroll:   0%|          | 1/343 [00:01<08:28,  1.49s/ lines]

Extracting prosody embedding for 1272-128104-0002 in Dev Enroll from /content/drive/MyDrive/INF2008_Machine_Learning/B3/data/libri_dev_enrolls_B3/wav/1272-128104-0002.wav


Processing Dev Enroll:   1%|          | 2/343 [00:04<12:33,  2.21s/ lines]

...

Extracting prosody embedding for 1272-135031-0006 in Dev Male Trial from /content/drive/MyDrive/INF2008_Machine_Learning/B3/data/libri_dev_trials_m_B3/wav/1272-135031-0006.wav


Processing Dev Male Trial:   1%|          | 7/960 [00:24<52:52,  3.33s/ lines]

Extracting prosody embedding for 1272-135031-0007 in Dev Male Trial from /content/drive/MyDrive/INF2008_Machine_Learning/B3/data/libri_dev_trials_m_B3/wav/1272-135031-0007.wav


Processing Dev Male Trial:   1%|          | 7/960 [00:28<1:05:33,  4.13s/ lines]


KeyboardInterrupt: 

In [ ]:
# Iterate over each category and folder in tacotron_folders
for category, folder in tacotron_folders.items():
    # Ensure the folder exists (it should already, but this is a double-check)
    os.makedirs(folder, exist_ok=True)
    # Count the number of files in the folder
    file_count = len(os.listdir(folder))
    # Print the category and file count
    print(f"{category}: {file_count} files")

Tacotron Dev Enroll files: 0
Tacotron Dev Male trial files: 0
Tacotron Dev Female trial files: 0
Tacotron Test Enroll files: 0
Tacotron Test Male trial files: 0
Tacotron Test Female trial files: 0
